In [27]:
import math
import torch
from torch import nn
from torchvision import models
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transforms, DataLoader, DataSet

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                          std=[0.229, 0.224, 0.225])
])

target_transform = transforms.Compose([
    transforms.Lambda(lambda y: torch.zeros(4, dtype=torch.float) \
      .scatter_(0, torch.tensor(y), value=1))
])

In [3]:
data_dir = '../data/images'
raw_data = datasets.ImageFolder(data_dir, 
                                transform=transform, 
                                target_transform=target_transform)
classes = raw_data.classes

In [4]:
sz = len(raw_data)
train_sz = math.floor(.8 * sz)
val_sz = sz - train_sz

In [5]:
train_dataset, val_dataset = random_split(raw_data, [train_sz, val_sz])

# Model Arch

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [28]:
classes = 4

# Define model
class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel, self).__init__()
        
        self.xfer = models.vgg11(pretrained=True)
        self.fc1 = nn.Linear(1000, classes)

    def forward(self, x):
        x = F.relu(self.xfer(x))
        return F.softmax(self.fc1(x), dim=1)
        
model = VisionModel().to(device)
print(model)

VisionModel(
  (xfer): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace=True)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace=True)
      (15): MaxPool2d(kernel_si

# Training

In [53]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    current = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        current += len(X)
        print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}], {batch}")

In [54]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [55]:
epochs = 5
batch_size = 32
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(val_dataset, batch_size=batch_size)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 0.000201  [   32/  291], 0
loss: 0.000463  [   64/  291], 1
loss: 0.000254  [   96/  291], 2
loss: 0.001493  [  128/  291], 3
loss: 0.000140  [  160/  291], 4
loss: 0.000315  [  192/  291], 5
loss: 0.000214  [  224/  291], 6
loss: 0.000114  [  256/  291], 7
loss: 0.000179  [  288/  291], 8
loss: 0.000002  [  291/  291], 9
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000472 

Epoch 2
-------------------------------
loss: 0.000136  [   32/  291], 0
loss: 0.000475  [   64/  291], 1
loss: 0.000188  [   96/  291], 2
loss: 0.001364  [  128/  291], 3
loss: 0.000457  [  160/  291], 4
loss: 0.000262  [  192/  291], 5
loss: 0.000134  [  224/  291], 6
loss: 0.000785  [  256/  291], 7
loss: 0.000163  [  288/  291], 8
loss: 0.000043  [  291/  291], 9
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000465 

Epoch 3
-------------------------------
loss: 0.000179  [   32/  291], 0
loss: 0.000341  [   64/  291], 1
loss: 0.001495  [   96/  291], 2
loss: 0.001027 